<h1> <center> Table of Contents </center> </h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#1.-Importing-Required-Libraries-and-Data"> Importing Required Libraries and Data</a></li>
        <li><a href="#2.-Data-Pre-processing-and-Preparation"> Data Pre-processing and Preparation</a></li>
        <li><a href="#3.-Different-Methods">Different Methods</a> </li>
        <li><a href="#4.-Summary-of-Different-Methods">Summary of Different Methods</a></li>                 
      </li>
    </ol>
</div>

# 1. Importing Required Libraries and Data

In [1]:
# !pip install python-Levenshtein
# !pip install "fuzzywuzzy==0.18.0"
# !pip install rapidfuzz
# !pip install spacy
# !pip install gensim
# !pip install python-Levenshtein
# !pip install rank_bm25


# !python -m spacy download en_core_web_lg 
# !pip install fast-autocomplete



In [1]:
# pip installs
# pip install python-Levenshtein
# pip install "fuzzywuzzy==0.18.0"
# pip install rapidfuzz
# pip install spacy
# pip install gensim
# !pip install python-Levenshtein
# from rank_bm25 import BM25Okapi
# nltk.download('punkt')
# nltk.download('stopwords') 
# nltk.download('wordnet')
# !pip install rank_bm25
# !python -m spacy download en_core_web_lg 
# !pip install fast-autocomplete
# pip install python-levenshtein



from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import rapidfuzz
from rapidfuzz import process, utils

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
# nltk.download('punkt')
# nltk.download('stopwords') 
# nltk.download('wordnet')
from rank_bm25 import BM25Okapi



import pandas as pd
from pandas.core.common import flatten

import timeit
import time

In [3]:
import numpy as np

In [4]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

In [5]:
#importing the file

dfdashboards = pd.read_csv('microstrategy_and_dashboard.csv')
dfmetrics = pd.read_csv('tableau_metric.csv')
dfdashboards.head()

,id,dashboards,caption,datasource,url,Global_usage,users_id_1,user_id_2
0,139444,Ad Analysis,Beer Style Level Value|Beer Style Levels|Beer ...,IRI 52wk 34.0,https:andanddashboards.armadilloinsight.comand...,1717,532,1185
1,139443,Ad Recap,Beer Style Level Value|Beer Style Levels|Beer ...,IRI 52wk 34.0,https:andanddashboards.armadilloinsight.comand...,974,523,451
2,139434,Aggregate Sales per Pt vs. CWD,Beer Style Level Value|Beer Style Levels|Beer ...,IRI 52wk 34.0|IRI Agg 34.0,https:andanddashboards.armadilloinsight.comand...,851,422,429
3,139466,Brand Control,Brand Family|Brand Levels|Brand|Main Brand Fam...,IRI 52wk 34.0,https:andanddashboards.armadilloinsight.comand...,923,625,298
4,139465,Brand In and Out Scorecard,Segment|Time as Date,IRI 52wk 34.0,https:andanddashboards.armadilloinsight.comand...,876,520,356


In [6]:
#converting column values to lowercase

dfdashboards = dfdashboards.apply(lambda x: x.astype(str).str.lower())
dfmetrics = dfmetrics.apply(lambda x: x.astype(str).str.lower())

In [7]:
#converting global usage to numeric

dfdashboards.Global_usage = pd.to_numeric(dfdashboards.Global_usage)

In [8]:
#splitting caption and descriptors columns 

dfdashboards = dfdashboards[['dashboards','caption','Global_usage']]
dfdashboards['metric_names'] = dfdashboards.caption.str.split('|')
dfmetrics['descriptors'] = dfmetrics.descriptors.str.split('|')

In [9]:
#converting metric names and descriptors in each row values to multiple rows

temp1 = dfdashboards.set_index(['dashboards'])['metric_names'].apply(pd.Series).stack().reset_index().drop('level_1', axis=1).rename(columns={0:'caption'})
temp2 = dfmetrics.set_index(['metric_name'])['descriptors'].apply(pd.Series).stack().reset_index().drop('level_1', axis=1).rename(columns={0:'descriptor'})

In [10]:
#merging metrics and descriptors

master = temp1.merge(temp2, how = 'inner', left_on ='caption', right_on = 'metric_name')[['dashboards','metric_name','descriptor']]
master = master.rename(columns={"dashboards":"dashboard_names", "metric_name": "metric_names", "descriptor": "descriptor_names"})
master

,dashboard_names,metric_names,descriptor_names
0,ad analysis,beer style level value,macro style value
1,ad analysis,beer style level value,mezzo style value
2,ad analysis,beer style level value,micro style valu
3,ad analysis,beer style level value,beer style value
4,ad analysis,beer style level value,altbier
...,...,...,...
236553,surveyed stores,surveyed stores,2018
236554,surveyed stores,surveyed stores,2019
236555,surveyed stores,surveyed stores,2020
236556,surveyed stores,surveyed stores,2021


In [11]:
#checking for metric and descriptors

master[(master['dashboard_names'] == '') | (master['descriptor_names'] == 'Apple ') | (master['metric_names'] == '')].head()

,dashboard_names,metric_names,descriptor_names


### Creating Corpus

In [12]:
#creating corpus

dashboard_names_list = list(master.dashboard_names.unique())
metric_names_list = list(master.metric_names.unique())
descriptor_names_list = list(master.descriptor_names.unique())
mastercorpus = dashboard_names_list + metric_names_list + descriptor_names_list

In [13]:
#pre-processing corpus

mastercorpus = list(filter(None,mastercorpus))
mastercorpus

['ad analysis',
 'ad recap',
 'aggregate sales per pt vs. cwd',
 'brands on ad',
 'brewery comparisons',
 'company ',
 'company brands over time',
 'r geography',
 'r geography over time',
 'company',
 'category and segment analysis',
 'competitive set',
 'dimensions over time',
 'dollar rank comp',
 'dollar ranker with cwd',
 'dollar vs. ad share',
 'dollars vs. growth',
 'feature ad count',
 'footprint',
 'line geogs over time',
 'merch efficiency',
 'merch efficiency c&amp;r',
 'merch execution',
 'merch ranker',
 'merch vs. non-merch',
 'our brands',
 'our brands dollar ranker',
 'package analysis',
 'price analysis',
 'price bucket analysis',
 'quick scorecard',
 'rankers',
 'sales per pt vs. cwd',
 'style analysis',
 'topline',
 'velocity',
 'volume rank comp',
 'market index analysis',
 'market index analysis visual',
 'price promotion count',
 'the humbler',
 'brand control',
 'item audit',
 'market share',
 'our packages',
 'depletions month cube with inventory and shipmentmen

# Fuzzywuzzy versions


## 3.2.1 fuzzywuzzy String Matching

FuzzyWuzzy is a library of Python which is used for string matching. Fuzzy string matching is the process of finding strings that match a given pattern. Basically it uses <b>Levenshtein Distance</b> to calculate the differences between sequences.

### Fuzzywuzzy Scorer Methods:
1. ratio
- partial_ratio
- token_sort_ratio
- token_set_ratio

In [14]:
s1 = 'cambridge woods drive 3742'
s2 = '3742 cambridge woods drive drive'

print("FuzzyWuzzy Ratio: ", fuzz.ratio(s1, s2)) 
print("FuzzyWuzzy partial_ratio: ", fuzz.partial_ratio(s1, s2)) 
print("FuzzyWuzzy token_sort_ratio: ", fuzz.token_sort_ratio(s1, s2)) 
print("FuzzyWuzzy token_set_ratio: ", fuzz.token_set_ratio(s1, s2)) 


FuzzyWuzzy Ratio:  76
FuzzyWuzzy partial_ratio:  85
FuzzyWuzzy token_sort_ratio:  90
FuzzyWuzzy token_set_ratio:  100


In [15]:
def fuzzywuzzy_suggestion(corpus, input_query):
   
    for i in [corpus]:
        
        return process.extract(input_query, i, limit=10)

def dashboard_names_suggestion(master, suggestions):
   
    dashboard_names = []
       
    for i in suggestions:
       
        if i in dashboard_names_list:
            
            dashboard_names.append(i)
        
        elif i in metric_names_list:
            
            dashboard_names = dashboard_names + (master.loc[(master.metric_names.str.lower() == i)].dashboard_names).to_list()
        
        else:
            
            dashboard_names = dashboard_names + (master.loc[(master.descriptor_names.str.lower() == i)].dashboard_names).to_list()

    return dashboard_names


In [16]:
# Input query:

InputQuery = 'Domestic cider  bottle'
# fuzzywuzzy search output

fuzzywuzzySuggestions = fuzzywuzzy_suggestion(mastercorpus,InputQuery)

# extract just the metric, descriptors, dashboards from output

fuzzywuzzySuggestionsList = []
for i in fuzzywuzzySuggestions:
    fuzzywuzzySuggestionsList.append(i[0])
print(fuzzywuzzySuggestionsList)
    
# extract dashboard names

fuzzywuzzyDashboardsList = list(dashboard_names_suggestion(master, fuzzywuzzySuggestionsList))

fuzzywuzzySearchOutput = []
fuzzywuzzySearchOutput = list(dict.fromkeys(fuzzywuzzyDashboardsList))
fuzzywuzzySearchOutput[:10]



['bottle', 'cider', '24 ct bottle', '1 ct long neck bottle 14.88 oz bottle', '1 ct bottle', 'appalachian mountain cider brand family', 'apple knocker hard cider brand family', 'cider creek hard cider brand family', 'cider house  of wisconsin brand family', 'cobbler mountain cider brand family']


['ad analysis',
 'aggregate sales per pt vs. cwd',
 'brands on ad',
 'brewery comparisons',
 'company brands over time',
 'r geography',
 'r geography over time',
 'company',
 'category and segment analysis',
 'competitive set']

## 3.2.2 Fuzzywuzzy using multiple scorers

In [17]:

ratio_sugg = {}
token_sort_ratio_sugg = {}
partial_ratio_sugg = {}
token_set_ratio_sugg = {}

def fuzzywuzzy_scorers_suggestions(User_Search, corpus):
    
    for token in corpus:
        sugg_score = fuzz.ratio(User_Search, token)
        ratio_sugg[token] = sugg_score
 
    
    for token in corpus:
        sugg_score = fuzz.partial_ratio(User_Search, token)
        partial_ratio_sugg[token] = sugg_score
   
    for token in corpus:
        sugg_score = fuzz.token_sort_ratio(User_Search, token)
        token_sort_ratio_sugg[token] = sugg_score

    
    for token in corpus:
        sugg_score = fuzz.token_set_ratio(User_Search, token)
        token_set_ratio_sugg[token] = sugg_score

    
        
def fuzzywuzzy_scorers_similiarity(scorers,suggestions_count,sort_scorers):
   
    scoreDf = pd.DataFrame()
    scoreDf = scoreDf.from_dict([ratio_sugg,partial_ratio_sugg, token_sort_ratio_sugg, token_set_ratio_sugg]).T.reset_index()
    scoreDf.columns = ['suggestion' ,'ratio_sugg','partial_ratio_sugg', 'token_sort_ratio_sugg','token_set_ratio_sugg']
    scoreDf['mean'] = scoreDf.mean(axis = 1)
    
    return scoreDf[scorers + ['mean']].sort_values(by = sort_scorers, ascending = False).head(suggestions_count)


In [18]:
input_query = 'Macro style values 11 CT 22 OZ bottle Dollar sales'
# scorers suggestion
fuzzywuzzy_scorers_suggestions(input_query.lower(), mastercorpus)

fuzzyScorers = ['suggestion' ,'ratio_sugg','partial_ratio_sugg', 'token_sort_ratio_sugg','token_set_ratio_sugg']
scorersSorting = ['ratio_sugg','partial_ratio_sugg', 'token_sort_ratio_sugg','token_set_ratio_sugg','mean']

# 'mean', ratio_sugg', 'partial_ratio_sugg','token_sort_ratio_sugg','token_set_ratio_sugg'
allScorersOutput = []

for sort_scorer in scorersSorting:
    
    fuzzywuzzySearchOutput = []

    scoreSortedDf = fuzzywuzzy_scorers_similiarity(fuzzyScorers, 15, sort_scorer)
    scoreSortedDf.sort_values(sort_scorer, ascending = False)

    fuzzywuzzyDashboardsList = list(dashboard_names_suggestion(master, scoreSortedDf['suggestion'][:10]))

    fuzzywuzzySearchOutput = list(dict.fromkeys(fuzzywuzzyDashboardsList))
    allScorersOutput.append(fuzzywuzzySearchOutput[:10])
    

scorersSuggestionsDf = pd.concat([d.reset_index(drop=True) for d in [
                        scoreSortedDf.sort_values(['ratio_sugg'], ascending = False)[['suggestion']],
                        scoreSortedDf.sort_values(['partial_ratio_sugg'], ascending = False)[['suggestion']],
                        scoreSortedDf.sort_values(['token_sort_ratio_sugg'], ascending = False)[['suggestion']],    
                        scoreSortedDf.sort_values(['token_set_ratio_sugg'], ascending = False)[['suggestion']],
                        scoreSortedDf.sort_values(['mean'], ascending = False)[['suggestion']]]],
                        axis=1)

scorersSuggestionsDf.columns  = ['ratio_sugg','partial_ratio_sugg', 'token_sort_ratio_sugg','token_set_ratio_sugg','mean']

print('User Search : ' + str(input_query))
display(scoreSortedDf)
display(scorersSuggestionsDf)

User Search : Macro style values 11 CT 22 OZ bottle Dollar sales


,suggestion,ratio_sugg,partial_ratio_sugg,token_sort_ratio_sugg,token_set_ratio_sugg,mean
496,11 ct 22 oz bottle,53,100,53,100,76.50
733,1 ct 22 oz bottle,51,100,51,94,74.00
514,12 ct 22 oz bottle,50,94,50,91,71.25
549,15 ct 22 oz bottle,50,94,50,91,71.25
564,2 ct 22 oz bottle,48,94,48,94,71.00
566,3 ct 22 oz bottle,48,94,45,94,70.25
630,4 ct 22 oz bottle,48,94,45,94,70.25
322,macro style value,51,100,51,79,70.25
664,6 ct 22 oz bottle,48,94,45,94,70.25
797,dollar sales,39,100,39,100,69.50


,ratio_sugg,partial_ratio_sugg,token_sort_ratio_sugg,token_set_ratio_sugg,mean
0,11 ct 22 oz bottle,11 ct 22 oz bottle,11 ct 22 oz bottle,11 ct 22 oz bottle,11 ct 22 oz bottle
1,1 ct 22 oz bottle,1 ct 22 oz bottle,1 ct 22 oz bottle,dollar sales,1 ct 22 oz bottle
2,macro style value,macro style value,macro style value,1 ct 22 oz bottle,12 ct 22 oz bottle
3,12 ct 22 oz bottle,dollar sales,12 ct 22 oz bottle,2 ct 22 oz bottle,15 ct 22 oz bottle
4,15 ct 22 oz bottle,12 ct 22 oz bottle,15 ct 22 oz bottle,3 ct 22 oz bottle,2 ct 22 oz bottle
5,2 ct 22 oz bottle,15 ct 22 oz bottle,2 ct 22 oz bottle,4 ct 22 oz bottle,3 ct 22 oz bottle
6,3 ct 22 oz bottle,2 ct 22 oz bottle,11 ct 7 oz bottle,6 ct 22 oz bottle,4 ct 22 oz bottle
7,4 ct 22 oz bottle,3 ct 22 oz bottle,1 ct 24 oz bottle,11 ct 7 oz bottle,macro style value
8,6 ct 22 oz bottle,4 ct 22 oz bottle,1 ct 20 oz bottle,12 ct 22 oz bottle,6 ct 22 oz bottle
9,11 ct 7 oz bottle,6 ct 22 oz bottle,1 ct 72 oz bottle,15 ct 22 oz bottle,dollar sales


In [19]:
column_names = ['ratio_sugg','partial_ratio_sugg', 'token_sort_ratio_sugg','token_set_ratio_sugg','mean']
finalSuggestionDF = pd.DataFrame(allScorersOutput).transpose().set_axis(column_names, axis =1)
display(finalSuggestionDF)

,ratio_sugg,partial_ratio_sugg,token_sort_ratio_sugg,token_set_ratio_sugg,mean
0,ad analysis,depletions month cube with inventory and shipm...,ad analysis,ad analysis,ad analysis
1,aggregate sales per pt vs. cwd,depletionsletions cube,aggregate sales per pt vs. cwd,r geography over time,aggregate sales per pt vs. cwd
2,brands on ad,retail month cube,brands on ad,category and segment analysis,brands on ad
3,brewery comparisons,retail cube to brand quality level,brewery comparisons,competitive set,brewery comparisons
4,company,retail cube to brand level,company,dimensions over time,company
5,company brands over time,retail cube with reorder to rollup,company brands over time,line geogs over time,company brands over time
6,r geography,sales team opportunitshis year,r geography,market share,r geography
7,r geography over time,sales manager dashboard,r geography over time,package analysis,r geography over time
8,company,depletionsletions,company,rankers,company
9,category and segment analysis,sales team tracker,category and segment analysis,style analysis,category and segment analysis


In [20]:
dfuserquey = pd.read_csv('sample_user_searchs.csv')



In [21]:
dfuserquey.columns

Index(['Dashboard', 'user_search'], dtype='object')

In [22]:
mydict = dict(zip(dfuserquey.user_search, dfuserquey.Dashboard))


In [80]:
# scorers suggestion
l = [] 
for input_query,expected_dashboard in mydict.items():
    fuzzywuzzy_scorers_suggestions(input_query.lower(), mastercorpus)
    
    fuzzyScorers = ['suggestion' ,'ratio_sugg','partial_ratio_sugg', 'token_sort_ratio_sugg','token_set_ratio_sugg']
    scorersSorting = ['ratio_sugg','partial_ratio_sugg', 'token_sort_ratio_sugg','token_set_ratio_sugg','mean']

    # 'mean', ratio_sugg', 'partial_ratio_sugg','token_sort_ratio_sugg','token_set_ratio_sugg'
    allScorersOutput = []

    for sort_scorer in scorersSorting:

        fuzzywuzzySearchOutput = []

        scoreSortedDf = fuzzywuzzy_scorers_similiarity(fuzzyScorers, 15, sort_scorer)
        scoreSortedDf.sort_values(sort_scorer, ascending = False)

        fuzzywuzzyDashboardsList = list(dashboard_names_suggestion(master, scoreSortedDf['suggestion'][:10]))

        fuzzywuzzySearchOutput = list(dict.fromkeys(fuzzywuzzyDashboardsList))
        allScorersOutput.append(fuzzywuzzySearchOutput[:10])


    scorersSuggestionsDf = pd.concat([d.reset_index(drop=True) for d in [
                            
                            scoreSortedDf.sort_values(['ratio_sugg'], ascending = False)[['suggestion']],
                            scoreSortedDf.sort_values(['partial_ratio_sugg'], ascending = False)[['suggestion']],
                            scoreSortedDf.sort_values(['token_sort_ratio_sugg'], ascending = False)[['suggestion']],    
                            scoreSortedDf.sort_values(['token_set_ratio_sugg'], ascending = False)[['suggestion']],
                            scoreSortedDf.sort_values(['mean'], ascending = False)[['suggestion']]]],
                            axis=1)

    scorersSuggestionsDf.columns  = ['ratio_sugg','partial_ratio_sugg', 'token_sort_ratio_sugg','token_set_ratio_sugg','mean']

    print('User Search : ' + str(input_query))
    print('Dashboard: ' + str(expected_dashboard).lower())

    
    column_names = ['ratio_sugg','partial_ratio_sugg', 'token_sort_ratio_sugg','token_set_ratio_sugg','mean']
    
    finalSuggestionDF = pd.DataFrame(allScorersOutput).transpose().set_axis(column_names, axis =1)
    

#     display(finalSuggestionDF)
    cols = ['ratio_sugg','partial_ratio_sugg', 'token_sort_ratio_sugg','token_set_ratio_sugg']
#     accuracy = pd.DataFrame(pd.np.empty((0, 5))).set_axis(cols,axis = 1)

    
    l.append(input_query)
#     for score in cols:

#         if expected_dashboard.strip().lower() in finalSuggestionDF[score].values:
#             inidex_match = finalSuggestionDF[score][finalSuggestionDF[score] == expected_dashboard.strip().lower()].index.tolist()
# #             print(inidex_match[0])
#             if inidex_match[0] == 0:
#                 l.append(1)
#                 print(1)
#             else: 
#                 l.append(1- 0.1*(inidex_match[0]+1))
#                 print(1- 0.1*(inidex_match[0]+1))
#         else:
#             l.append(0)
#             print("not found")
#     l.append(expected_dashboard)

    for score in cols:
        if expected_dashboard.strip().lower() in finalSuggestionDF[score].values:
            l.append(1)
        else:
            l.append(0)
    
    l.append(expected_dashboard)



User Search : Macro style values 11 CT 22 OZ bottle Dollar sales
Dashboard: ad analysis 
User Search : Volumn Sales 12 CT 1 Oz can alcoholic 2020
Dashboard: ad analysis 
User Search : Dollar Sales per Percent last year beer style Espiritus group 2018
Dashboard: ad analysis 
User Search : The fighting 69th Regiment 6/750 2020 beer styles
Dashboard: ad recap
User Search : Berentzen Apple Liqueur 2020 package type 
Dashboard: ad recap
User Search : Price per unit dollar sales IPA 12 CT 11 OZ can
Dashboard: dollar rank comp 
User Search : Vendor dollar sales New brands average items per store
Dashboard: dollar rank comp 
User Search : Pure holdings 12 CT 24 Oz bottles dollar trend last year 
Dashboard: dollar rank comp 
User Search : domestic amber ale micro style value 3.6 percent lift feature only segment 
Dashboard: brewery comparisons
User Search : imp-india ipa 6.9 percent import can 2-3 ct 14-18 oz 
Dashboard: brewery comparisons
User Search : 2 silos brewing co hi wire brand family 

In [81]:
output_columns = ['input_query','ratio_sugg','partial_ratio_sugg', 'token_sort_ratio_sugg','token_set_ratio_sugg','expected_dashboard']
output_df = pd.DataFrame(np.reshape(l, (-1, 6)),columns=output_columns)


In [82]:
output_df

,input_query,ratio_sugg,partial_ratio_sugg,token_sort_ratio_sugg,token_set_ratio_sugg,expected_dashboard
0,Macro style values 11 CT 22 OZ bottle Dollar s...,1,0,1,1,Ad Analysis
1,Volumn Sales 12 CT 1 Oz can alcoholic 2020,1,1,1,1,Ad Analysis
2,Dollar Sales per Percent last year beer style ...,1,1,1,1,Ad Analysis
3,The fighting 69th Regiment 6/750 2020 beer styles,1,0,1,0,Ad Recap
4,Berentzen Apple Liqueur 2020 package type,0,0,1,1,Ad Recap
5,Price per unit dollar sales IPA 12 CT 11 OZ can,1,0,0,1,Dollar Rank Comp
6,Vendor dollar sales New brands average items p...,0,0,0,1,Dollar Rank Comp
7,Pure holdings 12 CT 24 Oz bottles dollar trend...,1,0,1,0,Dollar Rank Comp
8,domestic amber ale micro style value 3.6 perce...,0,1,0,1,Brewery Comparisons
9,imp-india ipa 6.9 percent import can 2-3 ct 14...,0,1,1,1,Brewery Comparisons


In [83]:
output_df[output_df.columns[1:5]].apply(pd.to_numeric, errors='coerce').sum()

ratio_sugg               12
partial_ratio_sugg        5
token_sort_ratio_sugg    15
token_set_ratio_sugg     17
dtype: int64

## 3.2.3 Fuzzywuzzy using multiple suggestions 

In [28]:

input_query = 'Amber Ale micro style value 5.2 percent vendor can'

fuzzywuzyy_scorers_suggestion(input_query.lower(), mastercorpus)

fuzzyScorers = ['suggestion' ,'ratio_sugg','partial_ratio_sugg', 'token_sort_ratio_sugg','token_set_ratio_sugg']

scorersSorting = ['token_sort_ratio_sugg']

# 'mean', ratio_sugg', 'partial_ratio_sugg','token_sort_ratio_sugg','token_set_ratio_sugg'

allScorersOutput = []

for sort_scorers in scorersSorting:
    
    fuzzywuzzySearchOutput = []

    scoreSortedDf = fuzzywuzzy_scorers_similiarity(fuzzyScorers, 10, sort_scorers)

    fuzzywuzzyDashboardsList = list(dashboard_names_suggestion(master, scoreSortedDf['suggestion'][:10]))

    fuzzywuzzySearchOutput = list(dict.fromkeys(fuzzywuzzyDashboardsList))
    allScorersOutput.append(fuzzywuzzySearchOutput[:10])

scoreSortedDf = scoreSortedDf.sort_values(['mean'], ascending = False).head(8)
display(scoreSortedDf)

NameError: name 'fuzzywuzyy_scorers_suggestion' is not defined

In [ ]:
master[master['descriptor_names'].isin(list(scoreSortedDf['suggestion']))].\
                                    groupby(['dashboard_names'], as_index = False).agg('count').\
                                    sort_values( by = 'descriptor_names' , ascending = False)


In [ ]:
pd.set_option('display.max_colwidth', None)

master[master['dashboard_names'] == 'Merch vs. Non-Merch'.lower()][['descriptor_names']].head(100)

In [ ]:
masterNew[masterNew['dashboard_names'].isin(fuzzywuzzySearchOutput[:10])]

In [ ]:
dfdashboards[dfdashboards['dashboards'].isin(fuzzywuzzySearchOutput[:10])][['dashboards','caption']]